<a href="https://colab.research.google.com/github/kateburovova/dehumanization/blob/mainbranch/finetune_RuBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df_labels = pd.read_csv('/content/final_labels.csv', index_col=[0])

In [3]:
df_mention = df_labels[['text', 'Mention']]
df_emotion = df_labels[['text', 'Emotion']]

In [4]:
mention_label_mapping = {'так': 1, 'ні': 0, 'не можу визначитись із відповіддю': -1}
df_mention['labels'] = df_mention['Mention'].map(mention_label_mapping)

<ipython-input-4-1554e77b6fff>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mention['labels'] = df_mention['Mention'].map(mention_label_mapping)


In [5]:
# dropping lines where labelers are unsure
df_mention = df_mention[df_mention['labels']!=-1]
len(df_mention)

3518

In [6]:
emotion_label_mapping = {'так, присутня негативна': 1,
                         'ні, оцінка не присутня': 0,
                         'не можу визначитись з правильною відповіддю': -1,
                         'так, присутня позитивна': 2}
df_emotion['labels'] = df_emotion['Emotion'].map(emotion_label_mapping)

<ipython-input-6-b63af5414c05>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emotion['labels'] = df_emotion['Emotion'].map(emotion_label_mapping)


In [7]:
df_emotion = df_emotion[df_emotion.labels!=-1] #dropping those regarding which labelers are unsure

In [8]:
len(df_emotion)


3495

In [9]:
pip install wandb


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 10.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=c045bdf1e85aa52b38a7292c5127100108a8ffaf91cb26f42afbc8082a3fca77
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [10]:
import wandb

In [11]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.3 MB/s eta 0:00:00


In [23]:
import pandas as pd
import torch
import random
from transformers import set_seed
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

In [24]:
def set_random_seeds(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    set_seed(seed)  

seed = 42
set_random_seeds(seed)

# Mention

In [13]:
train_data, val_data = train_test_split(df_mention, test_size=0.2, random_state=42)
max_length = 512

In [14]:
tokenizer = BertTokenizerFast.from_pretrained("DeepPavlov/rubert-base-cased")
train_encodings = tokenizer(train_data["text"].tolist(), padding=True, truncation=True, max_length=max_length)
val_encodings = tokenizer(val_data["text"].tolist(), padding=True, truncation=True, max_length=max_length)

In [15]:
class BinaryClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = BinaryClassificationDataset(train_encodings, train_data["labels"].tolist())
val_dataset = BinaryClassificationDataset(val_encodings, val_data["labels"].tolist())

In [16]:
model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=2)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [26]:
# Load the pre-trained model
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate = 5e-5,
    load_best_model_at_end=True,
    weight_decay=0.01,
    metric_for_best_model="f1_score",
    report_to="wandb",
)

def compute_accuracy(predictions, labels):
    preds = predictions.argmax(-1)
    return {"accuracy": (preds == labels).mean()}

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     return compute_accuracy(predictions, labels)

def compute_f1_score(predictions, labels):
    preds = predictions.argmax(-1)
    return f1_score(labels, preds, average="weighted")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"f1_score": compute_f1_score(predictions, labels)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [27]:
wandb.init(project="binary_sentence_classification", name="rubert_mention_finetuning_colab")


wandb: Currently logged in as: kate-burovova (brvva_team). Use `wandb login --relogin` to force relogin


In [28]:
# Start training
best_model_TrainOutput = trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score
1,0.078500,0.359001,0.949725
2,0.000300,0.320047,0.958571


In [29]:
best_model_TrainOutput

TrainOutput(global_step=704, training_loss=0.03006206781190651, metrics={'train_runtime': 150.8492, 'train_samples_per_second': 37.309, 'train_steps_per_second': 4.667, 'total_flos': 1304366890281840.0, 'train_loss': 0.03006206781190651, 'epoch': 2.0})

In [25]:
wandb.finish()

eval/f1_score,▄█▅▄▆▇▆▃▅▆▆▆█▇▇▇▆▇▇▇▆▇▇▆▁▁▆▆▆▆
eval/loss,▁▁▃▄▂▃▃▄▄▄▃▃▂▃▃▃▃▃▄▄▄▄▄▄██▅▅▅▅
eval/runtime,▄▅▆▃▄▅▄▃▅▄▄▂▁▁▁▂▂▁▃▂▁▁▄▃▃▄█▃▂▂
eval/samples_per_second,▅▄▃▆▅▄▅▆▄▅▅▇███▇▇█▆▇██▅▆▆▅▁▆▇▇
eval/steps_per_second,▅▄▃▆▅▄▅▆▄▅▅▇███▇▇█▆▇██▅▆▆▅▁▆▇▇
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,▂▄█▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


In [30]:
trained_model = trainer.model

In [31]:
output_directory = "./saved_model"
trained_model.save_pretrained(output_directory)

#Emotion

In [32]:
train_data, val_data = train_test_split(df_emotion, test_size=0.2, random_state=42)
max_length = 512

In [35]:
tokenizer = BertTokenizerFast.from_pretrained("DeepPavlov/rubert-base-cased")

In [36]:
train_encodings = tokenizer(train_data["text"].tolist(), padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(val_data["text"].tolist(), padding=True, truncation=True, max_length=512)

In [38]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_data['labels'].tolist())
val_dataset = CustomDataset(val_encodings, val_data['labels'].tolist())

In [ ]:
model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=3)

In [46]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate = 5e-5,
    load_best_model_at_end=True,
    weight_decay=0.01,
    metric_for_best_model="f1_score",
    report_to="wandb",
)

def compute_f1_score(predictions, labels):
    preds = predictions.argmax(-1)
    return f1_score(labels, preds, average="weighted")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"f1_score": compute_f1_score(predictions, labels)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [48]:
wandb.init(project="binary_sentence_classification", name="rubert_emotion_finetuning_colab")

In [49]:
training_output = trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score
1,0.258500,0.331902,0.881072
2,0.220500,0.453910,0.879876
3,0.369600,0.471255,0.897502
4,0.085000,0.636877,0.887169
5,0.107300,0.705696,0.886770
6,0.020500,0.923383,0.869845
7,0.181100,1.157543,0.849368
8,0.302700,1.285658,0.811222
9,0.002300,0.929242,0.876769
10,0.000100,0.948116,0.882435


In [ ]:
wandb.finish()

We could either stop at 1 epoch or try more regularization apart from the weitghs decay that we already use.

In [60]:
from transformers import AutoModelForSequenceClassification, AutoConfig

config = AutoConfig.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=3, hidden_dropout_prob=0.5)
model = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased", config=config)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [61]:
from datetime import datetime

current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
run_name = f"run_{current_time}"

In [62]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate = 5e-5,
    load_best_model_at_end=True,
    weight_decay=0.01,
    metric_for_best_model="f1_score",
    report_to="wandb",
    run_name = run_name
)

def compute_f1_score(predictions, labels):
    preds = predictions.argmax(-1)
    return f1_score(labels, preds, average="weighted")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"f1_score": compute_f1_score(predictions, labels)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


In [63]:
wandb.init(project="binary_sentence_classification", name="rubert_emotion_finetuning_colab_dropout_0.5")

In [64]:
training_output = trainer.train()
wandb.finish()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Score
1,0.504500,0.553233,0.835862
2,0.276300,0.458917,0.872658
3,0.135100,0.516601,0.880175
4,0.243400,0.513834,0.898909
5,0.325500,0.511086,0.898831


eval/f1_score,▁▅▆██
eval/loss,█▁▅▅▅
eval/runtime,▆█▅▁▅
eval/samples_per_second,▃▁▄█▄
eval/steps_per_second,▃▁▄█▄
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▇▇▇▆▆▇▄▃▅▆▄▅▃▆▅▄▃▄▃▅▅▄▃▃▂▂▃▃▃▃▄▄▃▄▃▁▃▂▄
train/total_flos,▁
train/train_loss,▁


In [65]:
trained_model = trainer.model

In [66]:
output_directory = "./saved_2model"
trained_model.save_pretrained(output_directory)